Purpose

The purpose of this data science project is to come up with a pricing model for the the rents charged for german apartments. The project aims to build a predictive model for rents based on facilities and properties boasted by each individual listing. The model may be useful to companies or individuals looking to price their apartments for rents. 

Imports

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import requests
import pandas_profiling
from pandas_profiling.utils.cache import cache_file



Data Wrangling Objectives

Fundamental questions to resolve in this notebook before moving on.
Is this the right type of data that should be used to model rent prices?
    Has the required target feature been identified?
    Are there potentially useful features?
Are there fundamental issues with the data?

Loading the raw data

In [92]:
immo_data= pd.read_csv('../data/raw/immo_data.csv')

Data Definition

The features of an apartment (the entity) has to be defined in sufficient detail for further analysis. In doing so we gain an understanding of its relevance to the each apartment record. We also identify our feature of interest - the target feature - which in our case is the rent charged for each apartment, and start to get a sense for how these features are related to our target feature. Of relevance is representing the features in the right data format for further processing and taking note of features that have limited values. 

We start inspecting the dataframe by reviewing summary level information about the features and viewing the first view entries

This report gives us all the summary level information we want about the data set in a lot of detail

In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
report = immo_data.profile_report(sort='None', html={'style':{'full_width': True}}, progress_bar=False)


The info command gives a sense to the shape and quality of the dataframe.

In [95]:
immo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   regio1                    268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   picturecount              268850 non-null  int64  
 8   pricetrend                267018 non-null  float64
 9   telekomUploadSpeed        235492 non-null  float64
 10  totalRent                 228333 non-null  float64
 11  yearConstructed           211805 non-null  float64
 12  scoutId                   268850 non-null  int64  
 13  noParkSpaces              93052 non-null   f

We observe that the dataframe has 49 features associated with 268,850 apartment records that have been put up for rent. The dataframe features comprise of 4 main datatypes, including 6 boolean, 18 float, 6 integer and 19 object datatypes. We would need to verfiy that these datatypes are accurate in the context of our goal of developing a price model for rent. We also note a varying level of quality associated with each feature in terms of how many records exist for the particular feature. We would need to deal with those features that have a lot of empty values. One last observation is the amount of memory used up by the dataframe (89.7+ MB).This size is manageable with the operating system currently in use. A larger dataframe would necessitate the use of additional memory, possibly with cloud computing technologies. 

In [96]:
immo_data.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.0,1965.0,96107057,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.0,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,111378734,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.0,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.0,2019.0,113147523,1.0,NaN,False,Sachsen,True,9.0,965.0,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,108890903,NaN,district_heating,False,Sachsen,False,2.0,343.0,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.0,1950.0,114751222,NaN,gas,False,Bremen,False,1.0,765.0,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


The head method alone allows us to see some, but not all the features of the dataframe. We need additional code to view all the features. 

In [97]:
pd.options.display.max_columns = None
immo_data.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.0,1965.0,96107057,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.0,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,111378734,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.0,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.0,2019.0,113147523,1.0,NaN,False,Sachsen,True,9.0,965.0,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,108890903,NaN,district_heating,False,Sachsen,False,2.0,343.0,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.0,1950.0,114751222,NaN,gas,False,Bremen,False,1.0,765.0,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


We observe a variety of prices, costs and factors that may be used to determine rent. These includes serviceCharge, priceTrend, totalRent, baseRent, baseRentRange, heatingCosts, electricityBasePrice, electricityKwhPrice. We also notice that there appears to be a repeat of regional information. We also notice a good amount of text data in german that requires translation to english. Let's note the different features of the dataframe. 

Explore the Data

First of all we need to identify each record uniquely. It would be preferrable not to use address information captured in the street and streetPlain feature since this involves using 2 feature and the datatype of these feature is an object (text), which makes it difficult for processing. However, there is scoutID feature that may work. This feature must have unique values for all 268,850 apartment records. 

In [98]:
immo_data["scoutId"].nunique()

268850

Perfect! Each scoutId value is distinct and exists for all 268,850 records. 

Let's now move scoutId to the first column as a unique identifier for each record. 

In [99]:
cols = list(immo_data) #Get a list of column names for the dataframe
#Get the list index of scoutId using list index method, pop it from the list using the list pop method, and insert the value (scoutId) as the first list item using the list insert method
cols.insert(0,cols.pop(cols.index("scoutId")))
#Use the loc indexer to call the dataframe with the new column arrangement and assign to the original dataframe
immo_data = immo_data.loc[:, cols]

In [100]:
pd.options.display.max_columns = 6
immo_data.head(2)

,scoutId,regio1,serviceCharge,...,electricityBasePrice,electricityKwhPrice,date
0,96107057,Nordrhein_Westfalen,245.0,...,NaN,NaN,May19
1,111378734,Rheinland_Pfalz,134.0,...,NaN,NaN,May19


scoutId is now the first feature that identifies each apartment record

In [101]:
pd.options.display.max_columns = None
immo_data.head()

,scoutId,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,96107057,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.0,1965.0,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.0,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,111378734,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.0,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,113147523,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.0,2019.0,1.0,NaN,False,Sachsen,True,9.0,965.0,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,108890903,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,NaN,district_heating,False,Sachsen,False,2.0,343.0,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,114751222,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.0,1950.0,NaN,gas,False,Bremen,False,1.0,765.0,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


Number of missing values per feature

In [102]:
# To create a dataframe with  %age of values that are missing
missing = pd.concat([immo_data.isnull().sum(), 100 * immo_data.isnull().mean()], axis = 1)
missing.columns = ['count', '%']
missing.sort_values(by = 'count').T

,scoutId,regio3,regio2,livingSpaceRange,garden,noRoomsRange,noRooms,geo_plz,baseRentRange,lift,street,geo_krs,livingSpace,baseRent,cellar,geo_bln,date,newlyConst,regio1,hasKitchen,picturecount,balcony,pricetrend,serviceCharge,description,telekomTvOffer,telekomUploadSpeed,typeOfFlat,totalRent,heatingType,floor,facilities,firingTypes,yearConstructed,yearConstructedRange,condition,streetPlain,houseNumber,numberOfFloors,thermalChar,interiorQual,petsAllowed,noParkSpaces,heatingCosts,lastRefurbish,energyEfficiencyClass,electricityKwhPrice,electricityBasePrice,telekomHybridUploadSpeed
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1832.000000,6909.000000,19747.000000,32619.000000,33358.000000,36614.000000,40517.000000,44856.000000,51309.00000,52924.000000,56964.000000,57045.000000,57045.000000,68489.0000,71013.000000,71018.000000,97732.000000,106506.000000,112665.000000,114573.000000,175798.000000,183332.000000,188139.000000,191063.000000,222004.000000,222004.000000,223830.000000
%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.681421,2.569834,7.344988,12.132788,12.407662,13.618747,15.070485,16.684397,19.08462,19.685326,21.188023,21.218151,21.218151,25.4748,26.413614,26.415473,36.351869,39.615399,41.906267,42.615957,65.388879,68.191185,69.979171,71.066766,82.575414,82.575414,83.254603


We notice that 27 features have missing values, with 7 features having over 50% of values missing. We also notice that while baseRent has no missing values, totalRent has over 15% of its values missing. We need to decide whether to use basRent over totalRent as our target feature. 

Categorical Features

We need to treat categorical features differently from numeric features. We first of all determine the categorical features starting with those having an object datatype. 

In [103]:
immo_cat = immo_data.select_dtypes('object')

In [104]:
immo_cat.head()

,regio1,heatingType,telekomTvOffer,firingTypes,geo_bln,houseNumber,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,typeOfFlat,regio2,regio3,description,facilities,energyEfficiencyClass,date
0,Nordrhein_Westfalen,central_heating,ONE_YEAR_FREE,oil,Nordrhein_Westfalen,244,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,ground_floor,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,May19
1,Rheinland_Pfalz,self_contained_central_heating,ONE_YEAR_FREE,gas,Rheinland_Pfalz,NaN,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,ground_floor,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,May19
2,Sachsen,floor_heating,ONE_YEAR_FREE,NaN,Sachsen,4,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,apartment,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,Oct19
3,Sachsen,district_heating,ONE_YEAR_FREE,district_heating,Sachsen,35,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,other,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,NaN,May19
4,Bremen,self_contained_central_heating,NaN,gas,Bremen,10,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,apartment,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,Feb20


In [105]:
immo_cat.shape

(268850, 19)

We have 19 objects

In [106]:
missing_cat = pd.concat([immo_cat.isnull().sum(), 100 * immo_cat.isnull().mean()], axis = 1)
missing_cat.columns = ['count', '%']
missing_cat.sort_values(by = 'count').T

,regio1,regio3,regio2,street,geo_krs,date,geo_bln,description,telekomTvOffer,typeOfFlat,heatingType,facilities,firingTypes,condition,streetPlain,houseNumber,interiorQual,petsAllowed,energyEfficiencyClass
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19747.000000,32619.000000,36614.000000,44856.000000,52924.000000,56964.000000,68489.0000,71013.000000,71018.000000,112665.000000,114573.000000,191063.000000
%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.344988,12.132788,13.618747,16.684397,19.685326,21.188023,25.4748,26.413614,26.415473,41.906267,42.615957,71.066766


Let's examine the features that have complete values

In [107]:
complete_cat = immo_cat.loc[:, immo_cat.isnull().sum() == 0]

In [108]:
complete_cat.head()

,regio1,geo_bln,geo_krs,street,regio2,regio3,date
0,Nordrhein_Westfalen,Nordrhein_Westfalen,Dortmund,Sch&uuml;ruferstra&szlig;e,Dortmund,Schüren,May19
1,Rheinland_Pfalz,Rheinland_Pfalz,Rhein_Pfalz_Kreis,no_information,Rhein_Pfalz_Kreis,Böhl_Iggelheim,May19
2,Sachsen,Sachsen,Dresden,Turnerweg,Dresden,Äußere_Neustadt_Antonstadt,Oct19
3,Sachsen,Sachsen,Mittelsachsen_Kreis,Gl&uuml;ck-Auf-Stra&szlig;e,Mittelsachsen_Kreis,Freiberg,May19
4,Bremen,Bremen,Bremen,Hermann-Henrich-Meier-Allee,Bremen,Neu_Schwachhausen,Feb20


The data owner noted that regio1 and geo_bln have the same values, and geo_krs and regio2 have the same values as well. We can test these and eliminate a redundant features. 

In [109]:
(complete_cat.regio1 != complete_cat.geo_bln).value_counts() # Number of records where regio1 is not equal to geo_bln

False    268850
dtype: int64

This implies that regio1 is always equal to geo_bln. We will drop geo_bln.

In [110]:
(complete_cat.geo_krs != complete_cat.regio2).value_counts() # Number of records where geo_krs and regio2 are not equal

False    268850
dtype: int64

This implies that regio2 and goe_krs values are always equal. We will drop geo_krs

In [111]:
complete_cat.drop(columns = ['geo_bln', 'geo_krs'], axis = 1, inplace = True)

C:\Users\zebin\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [112]:
complete_cat.columns #Verifying that geo_bln and geo_krs are gone from the complete_cat dataframe

Index(['regio1', 'street', 'regio2', 'regio3', 'date'], dtype='object')

In [113]:
immo_data.drop(columns = ['geo_bln', 'geo_krs'], axis = 1, inplace = True)

In [114]:
immo_data.select_dtypes("object").columns #Verifying that the geo_bln and geo_krs features are gone from the immo_data dataframe

Index(['regio1', 'heatingType', 'telekomTvOffer', 'firingTypes', 'houseNumber',
       'condition', 'interiorQual', 'petsAllowed', 'street', 'streetPlain',
       'typeOfFlat', 'regio2', 'regio3', 'description', 'facilities',
       'energyEfficiencyClass', 'date'],
      dtype='object')

In [115]:
complete_cat.head()

,regio1,street,regio2,regio3,date
0,Nordrhein_Westfalen,Sch&uuml;ruferstra&szlig;e,Dortmund,Schüren,May19
1,Rheinland_Pfalz,no_information,Rhein_Pfalz_Kreis,Böhl_Iggelheim,May19
2,Sachsen,Turnerweg,Dresden,Äußere_Neustadt_Antonstadt,Oct19
3,Sachsen,Gl&uuml;ck-Auf-Stra&szlig;e,Mittelsachsen_Kreis,Freiberg,May19
4,Bremen,Hermann-Henrich-Meier-Allee,Bremen,Neu_Schwachhausen,Feb20


Let's now examine the features with incomplete values

In [116]:
incomplete_cat = immo_cat.loc[:, immo_cat.isnull().sum() > 0]

In [117]:
incomplete_cat.columns

Index(['heatingType', 'telekomTvOffer', 'firingTypes', 'houseNumber',
       'condition', 'interiorQual', 'petsAllowed', 'streetPlain', 'typeOfFlat',
       'description', 'facilities', 'energyEfficiencyClass'],
      dtype='object')

In [118]:
incomplete_cat.shape

(268850, 12)

There are 12 incomplete categorical columns.

In [119]:
incomplete_cat.head()

,heatingType,telekomTvOffer,firingTypes,houseNumber,condition,interiorQual,petsAllowed,streetPlain,typeOfFlat,description,facilities,energyEfficiencyClass
0,central_heating,ONE_YEAR_FREE,oil,244,well_kept,normal,NaN,Schüruferstraße,ground_floor,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN
1,self_contained_central_heating,ONE_YEAR_FREE,gas,NaN,refurbished,normal,no,NaN,ground_floor,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN
2,floor_heating,ONE_YEAR_FREE,NaN,4,first_time_use,sophisticated,NaN,Turnerweg,apartment,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN
3,district_heating,ONE_YEAR_FREE,district_heating,35,NaN,NaN,NaN,Glück-Auf-Straße,other,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,NaN
4,self_contained_central_heating,NaN,gas,10,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,apartment,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN


Let's see if we can eliminate variables with a large number of empty values

We first of all determine the proportion of missing values for each variable

In [120]:
missing_incomplete_cat = pd.concat([incomplete_cat.isnull().sum(), 100 * incomplete_cat.isnull().mean()], axis = 1)
missing_incomplete_cat.columns = ['count', '%']
missing_incomplete_cat.sort_values(by = 'count').T

,description,telekomTvOffer,typeOfFlat,heatingType,facilities,firingTypes,condition,streetPlain,houseNumber,interiorQual,petsAllowed,energyEfficiencyClass
count,19747.000000,32619.000000,36614.000000,44856.000000,52924.000000,56964.000000,68489.0000,71013.000000,71018.000000,112665.000000,114573.000000,191063.000000
%,7.344988,12.132788,13.618747,16.684397,19.685326,21.188023,25.4748,26.413614,26.415473,41.906267,42.615957,71.066766


We first of all notice that energyEfficiencyClass has over 70% of its values missing. We are likely to delete this variable, but let's explore these incomplete categorical columns to determine if we can replace the null values. 

Let's review the number of unique values associated with the features

In [121]:
incomplete_cat.nunique()

heatingType                  13
telekomTvOffer                3
firingTypes                 132
houseNumber                5510
condition                    10
interiorQual                  4
petsAllowed                   3
streetPlain               54490
typeOfFlat                   10
description              212621
facilities               189526
energyEfficiencyClass        10
dtype: int64

Let's now get a sense for the values of features with distinct values that are less than 15

In [122]:
distinct = incomplete_cat.nunique() # Array of the number of distinct values

In [123]:
distinct_15 = distinct[distinct < 15] # Filter that shows the number of distinct values less than 15

In [124]:
distinct_15

heatingType              13
telekomTvOffer            3
condition                10
interiorQual              4
petsAllowed               3
typeOfFlat               10
energyEfficiencyClass    10
dtype: int64

In [125]:
index_15 = distinct_15.index # Index of distinct values greater than 15 corresponding to the column names

In [126]:
[[i, incomplete_cat[i].unique()] for i in index_15] # List that provides the feature and the names of unique values for the feature

[['heatingType',
  array(['central_heating', 'self_contained_central_heating',
         'floor_heating', 'district_heating', 'gas_heating', 'oil_heating',
         nan, 'wood_pellet_heating', 'electric_heating',
         'combined_heat_and_power_plant', 'heat_pump',
         'night_storage_heater', 'stove_heating', 'solar_heating'],
        dtype=object)],
 ['telekomTvOffer',
  array(['ONE_YEAR_FREE', nan, 'NONE', 'ON_DEMAND'], dtype=object)],
 ['condition',
  array(['well_kept', 'refurbished', 'first_time_use', nan,
         'fully_renovated', 'mint_condition',
         'first_time_use_after_refurbishment', 'modernized', 'negotiable',
         'need_of_renovation', 'ripe_for_demolition'], dtype=object)],
 ['interiorQual',
  array(['normal', 'sophisticated', nan, 'simple', 'luxury'], dtype=object)],
 ['petsAllowed', array([nan, 'no', 'negotiable', 'yes'], dtype=object)],
 ['typeOfFlat',
  array(['ground_floor', 'apartment', 'other', nan, 'roof_storey',
         'raised_ground_floor', '

As mentioned the energyEfficiencyClass variable has over 70% of its values missing. This variable has 10 distinct values corresponding to an energy efficiency rating that ranges  from an A_PLUS rating  to an F rating. The data owner noted that this information was deprecated as of the Feb 20 data pull. Furthermore,the data owner noted that this variable is based on the thermalCar variable. This means that we still have a variable that provides information on energy efficiency in the absence of energyEfficiencyClass. We will therefore drop this variable.  

In [127]:
immo_data.drop(columns = ['energyEfficiencyClass'], axis = 1, inplace = True)

In [128]:
immo_data.select_dtypes("object").columns #Verifying that the geo_bln and geo_krs features are gone from the immo_data dataframe

Index(['regio1', 'heatingType', 'telekomTvOffer', 'firingTypes', 'houseNumber',
       'condition', 'interiorQual', 'petsAllowed', 'street', 'streetPlain',
       'typeOfFlat', 'regio2', 'regio3', 'description', 'facilities', 'date'],
      dtype='object')

In [129]:
distinct_plus = distinct[distinct > 15] # Filter that shows the number of distinct values greater than 15

In [130]:
distinct_plus

firingTypes       132
houseNumber      5510
streetPlain     54490
description    212621
facilities     189526
dtype: int64

In [131]:
incomplete_cat.loc[:, distinct_plus.index].head(10)

,firingTypes,houseNumber,streetPlain,description,facilities
0,oil,244,Schüruferstraße,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...
1,gas,NaN,NaN,Alles neu macht der Mai – so kann es auch für ...,NaN
2,NaN,4,Turnerweg,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,..."
3,district_heating,35,Glück-Auf-Straße,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN
4,gas,10,Hermann-Henrich-Meier-Allee,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...
5,gas,1e,Hardeseiche,NaN,"helle ebenerdige 2 Zi. Wohnung mit Terrasse, h..."
6,NaN,14,Am_Bahnhof,Am Bahnhof 14 in Freiberg\nHeizkosten und Warm...,NaN
7,gas:electricity,35,Lesumer_Heerstr.,+ Komfortabler Bodenbelag: Die Wohnung ist zus...,Rollläden; Warmwasserbereiter; Kellerraum; Gas...
8,oil,NaN,NaN,"Diese ansprechende, lichtdurchflutete DG-Wohnu...","Parkett, Einbauküche, kein Balkon"
9,gas,30,Hüttenstr.,Sie sind auf der Suche nach einer gepflegten u...,In Ihrem neuen Zuhause können Sie nach wenigen...


In [132]:
incomplete_cat.firingTypes.unique()[:20]

array(['oil', 'gas', nan, 'district_heating', 'gas:electricity',
       'electricity', 'pellet_heating', 'natural_gas_light',
       'combined_heat_and_power_fossil_fuels',
       'district_heating:local_heating', 'steam_district_heating',
       'natural_gas_heavy', 'gas:district_heating', 'solar_heating:gas',
       'environmental_thermal_energy', 'local_heating',
       'gas:natural_gas_light', 'geothermal',
       'combined_heat_and_power_regenerative_energy', 'heat_supply'],
      dtype=object)

firingTypes is described by the data owner as the "main energy sources, separated by colon". This is somewhat similar to the heatingType feature described as the "Type of heating". We can determine how often these features have the same value.

In [133]:
(immo_data.firingTypes != immo_data.heatingType).value_counts()

True     249296
False     19554
dtype: int64

We note that the firingTypes and heatingTypes feature are typically not the same. As mentioned earlier on, the firingTypes include all the fuel sources separated by a column. We can separate these fuel sources and pick the first fuel source as the main fuel source in lieu of having a list. 

In [134]:
split_data = immo_data["firingTypes"].str.split(":", expand = True)

In [135]:
split_data.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12
268845,geothermal,None,None,None,None,None,None,None,None,None,None,None,None
268846,gas,None,None,None,None,None,None,None,None,None,None,None,None
268847,gas,None,None,None,None,None,None,None,None,None,None,None,None
268848,gas,None,None,None,None,None,None,None,None,None,None,None,None
268849,district_heating,None,None,None,None,None,None,None,None,None,None,None,None


We have a record with 13 possible firing types. But most entries have one. We'll give these firing types columns from 1 to 13

In [136]:
split_data.columns = ["fuelType1", "fuelType2", "fuelType3", "fuelType4", "fuelType5", "fuelType6", "fuelType7", "fuelType8", "fuelType9", "fuelType10", "fuelType11", "fuelType12", "fuelType13"]

In [137]:
fueltypes = [[x, split_data[x].count()] for x in ["fuelType1", "fuelType2", "fuelType3", "fuelType4", "fuelType5", "fuelType6", "fuelType7", "fuelType8", "fuelType9", "fuelType10", "fuelType11", "fuelType12", "fuelType13"]]

In [138]:
fueltypes

[['fuelType1', 211886],
 ['fuelType2', 3407],
 ['fuelType3', 69],
 ['fuelType4', 17],
 ['fuelType5', 4],
 ['fuelType6', 4],
 ['fuelType7', 2],
 ['fuelType8', 2],
 ['fuelType9', 2],
 ['fuelType10', 2],
 ['fuelType11', 2],
 ['fuelType12', 2],
 ['fuelType13', 1]]

There are more records in fuelType 1 than the others. We can select fuelType1 as the representive fuel for the firingTypes feature.

In [139]:
primaryFuel = split_data[["fuelType1"]]

In [140]:
immo_data.shape

(268850, 46)

In [141]:
immo_data = pd.concat([immo_data, primaryFuel], axis = 1)

In [142]:
immo_data.shape

(268850, 47)

In [143]:
immo_data.head()

,scoutId,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,noParkSpaces,firingTypes,hasKitchen,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,lastRefurbish,electricityBasePrice,electricityKwhPrice,date,fuelType1
0,96107057,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.0,1965.0,1.0,oil,False,True,2.0,595.0,244,86.00,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,May19,oil
1,111378734,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,2.0,gas,False,False,1.0,800.0,NaN,89.00,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,2019.0,NaN,NaN,May19,gas
2,113147523,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.0,2019.0,1.0,NaN,False,True,9.0,965.0,4,83.80,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,Oct19,NaN
3,108890903,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,NaN,district_heating,False,False,2.0,343.0,35,58.15,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,May19,district_heating
4,114751222,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.0,1950.0,NaN,gas,False,False,1.0,765.0,10,84.97,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,Feb20,gas


We can drop the original firingTypes column since we have captured relevant information in fuelType

In [144]:
immo_data.drop(columns = ["firingTypes"], axis = 1, inplace = True)

We can now rename fuelType1 as firingType

In [145]:
immo_data.rename(columns={'fuelType1':'firingType'}, inplace=True)


In [146]:
immo_data.columns

Index(['scoutId', 'regio1', 'serviceCharge', 'heatingType', 'telekomTvOffer',
       'telekomHybridUploadSpeed', 'newlyConst', 'balcony', 'picturecount',
       'pricetrend', 'telekomUploadSpeed', 'totalRent', 'yearConstructed',
       'noParkSpaces', 'hasKitchen', 'cellar', 'yearConstructedRange',
       'baseRent', 'houseNumber', 'livingSpace', 'condition', 'interiorQual',
       'petsAllowed', 'street', 'streetPlain', 'lift', 'baseRentRange',
       'typeOfFlat', 'geo_plz', 'noRooms', 'thermalChar', 'floor',
       'numberOfFloors', 'noRoomsRange', 'garden', 'livingSpaceRange',
       'regio2', 'regio3', 'description', 'facilities', 'heatingCosts',
       'lastRefurbish', 'electricityBasePrice', 'electricityKwhPrice', 'date',
       'firingType'],
      dtype='object')

In [147]:
(immo_data.firingType!= immo_data.heatingType).value_counts()

True     249238
False     19612
dtype: int64

We still observe that by splitting the firingTypes columns and choosing the first fuel type (now firingType), there is an increase in the number of values for which the heatingType is the same as the firingType; however, the diffence is not much. 

In [148]:
immo_data.heatingType.nunique(), immo_data.firingType.nunique()

(13, 25)

We have reduced the number of values for firingType from 132 to 25

In [149]:
immo_data[['heatingType', 'firingType']]

,heatingType,firingType
0,central_heating,oil
1,self_contained_central_heating,gas
2,floor_heating,NaN
3,district_heating,district_heating
4,self_contained_central_heating,gas
...,...,...
268845,heat_pump,geothermal
268846,gas_heating,gas
268847,central_heating,gas
268848,heat_pump,gas


In [150]:
heatingList = immo_data.heatingType.unique()

In [151]:
heatingList

array(['central_heating', 'self_contained_central_heating',
       'floor_heating', 'district_heating', 'gas_heating', 'oil_heating',
       nan, 'wood_pellet_heating', 'electric_heating',
       'combined_heat_and_power_plant', 'heat_pump',
       'night_storage_heater', 'stove_heating', 'solar_heating'],
      dtype=object)

In [152]:
firingList = immo_data.firingType.unique()

In [153]:
firingList

array(['oil', 'gas', nan, 'district_heating', 'electricity',
       'pellet_heating', 'natural_gas_light',
       'combined_heat_and_power_fossil_fuels', 'steam_district_heating',
       'natural_gas_heavy', 'solar_heating',
       'environmental_thermal_energy', 'local_heating', 'geothermal',
       'combined_heat_and_power_regenerative_energy', 'heat_supply',
       'liquid_gas', 'wood', 'hydro_energy',
       'combined_heat_and_power_renewable_energy', 'coal', 'bio_energy',
       'wood_chips', 'combined_heat_and_power_bio_energy', 'wind_energy',
       'coal_coke'], dtype=object)

Let's see if we can find common and different elements between these two lists

In [154]:
firingSet = set(firingList)

In [155]:
heatingSet = set(heatingList)

In [156]:
firingSet.intersection(heatingList) # Common values

{'district_heating', nan, 'solar_heating'}

In [157]:
heatingSet.difference(firingSet) # Values in heatingSet that are not in firingSet

{'central_heating',
 'combined_heat_and_power_plant',
 'electric_heating',
 'floor_heating',
 'gas_heating',
 'heat_pump',
 'night_storage_heater',
 'oil_heating',
 'self_contained_central_heating',
 'stove_heating',
 'wood_pellet_heating'}

In [158]:
firingSet.difference(heatingSet) # Values in firingSet that are not in heatingSet

{'bio_energy',
 'coal',
 'coal_coke',
 'combined_heat_and_power_bio_energy',
 'combined_heat_and_power_fossil_fuels',
 'combined_heat_and_power_regenerative_energy',
 'combined_heat_and_power_renewable_energy',
 'electricity',
 'environmental_thermal_energy',
 'gas',
 'geothermal',
 'heat_supply',
 'hydro_energy',
 'liquid_gas',
 'local_heating',
 'natural_gas_heavy',
 'natural_gas_light',
 'oil',
 'pellet_heating',
 'steam_district_heating',
 'wind_energy',
 'wood',
 'wood_chips'}

The values unique to the firingSet are mainly energy sources as noted earlier on. Of these, the environmental_thermal_energy, heat_supply and local_heating are unclear fuel types. district_heating and solar_heating could also be understood as fuel sources for the firingSet. district_heating could use a variety of fuel sources, while solar heating uses energy from the sun. The firingSet feature is therefore easily understood, except for  the aforementioned unclear fuel sources. Conversely, the values unique to the heatingSet are somewhat inconsistent in terms of what this feature represents. heatingSet comprises of heat generation systems (i.e. combined_heat_and_power_plant, electric_heating, night_storage_heater), heat distribution systems (i.e. central_heating, floor_heating, heat_pump, self_contained_central_heating, stove_heating) and fuel types (i.e. gas_heating, oil_heating, wood_pellet_heating). district_heating would be a heat generation system, while solar_heating would be a fuel type for the heatingSet. It would be difficult to establish a relationship between our target feature and the heatingType feature. Moreover, we still need to determine if any of these features matter in determining rent. Do landlords price apartments higher with renewable energy sources than non-renewable sources? We would attempt to answer this question. 

TV Offer: telekomTvOffer has four values - 'ONE_YEAR_FREE', nan, 'NONE', 'ON_DEMAND'. We can assume that "ON_DEMAND" is an apartment without a TV offer. We can convert all values related to 'NONE', 'ON_DEMAND' and nan to zero. 

In [159]:
immo_data.replace({'telekomTvOffer':{np.nan: 0, 'NONE': 0, 'ON_DEMAND': 0}}, inplace = True)

In [160]:
immo_data.head()

,scoutId,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,noParkSpaces,hasKitchen,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,lastRefurbish,electricityBasePrice,electricityKwhPrice,date,firingType
0,96107057,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.0,1965.0,1.0,False,True,2.0,595.0,244,86.00,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,May19,oil
1,111378734,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,2.0,False,False,1.0,800.0,NaN,89.00,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,2019.0,NaN,NaN,May19,gas
2,113147523,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.0,2019.0,1.0,False,True,9.0,965.0,4,83.80,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,Oct19,NaN
3,108890903,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,NaN,False,False,2.0,343.0,35,58.15,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,May19,district_heating
4,114751222,Bremen,138.00,self_contained_central_heating,0,NaN,False,True,19,2.46,NaN,903.0,1950.0,NaN,False,False,1.0,765.0,10,84.97,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,Feb20,gas


All values within the telekomTVOffer column should be available

In [161]:
immo_data.telekomTvOffer.isnull().sum() #values should add up to zero

0

street / streetPlain: Data owner noted that these two columns are the same but with different formatting. Let's see if these two columns have the same number of available values. 

In [162]:
(immo_data.street.isnull().sum(), immo_data.streetPlain.isnull().sum())

(0, 71013)

There is a difference in the number of values in street and streetPlain. However, in observing the data, we notice that unavailable information under street is referred to as no_information. Let's see if no_information in street is the same as nan in streetPlain.

In [164]:
immo_data.street.str.count('no_information').sum()

71013

This is the exact number that we get for nan in streetPlain. We can assume that missing values for both are the same. 

Which formatting is preffered between street and streetPlain. Let's observe a few records

In [166]:
immo_data[['street', 'streetPlain']].head(5)

,street,streetPlain
0,Sch&uuml;ruferstra&szlig;e,Schüruferstraße
1,no_information,NaN
2,Turnerweg,Turnerweg
3,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße
4,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee


streetPlain appears to be in a better format than street. We will go ahead and eliminate street. 

In [168]:
immo_data.drop(columns = 'street', axis = 1, inplace = True)

In [172]:
immo_data.shape

(268850, 45)

We note that we have an identifier for each apartment and therefore do not need a house number. We assume that we are likely to find relationships between rent and the street of the apartment rather than between the rent and the house number. We will go ahead and drop the houseNumber variable. 

In [176]:
immo_data.drop(columns = ['houseNumber'], axis = 1, inplace = True)

In [177]:
immo_data.shape

(268850, 44)

We observe that there are four values associated with whether pets are allowed: yes, no, negotiable, nan. We can convert all nan entries to negotiable, since a definite answer was not provided. 

In [181]:
immo_data.replace({'petsAllowed':{np.nan: 'negotiable'}}, inplace = True)

Numeric Features

Let's now examine the features that may have a relationship with rent - our target feature.

In [30]:
#Create a list and dataframe of the relevant rent features
rentList = ['totalRent','baseRent', 'serviceCharge', 'heatingCosts', 'electricityBasePrice', 'electricityKwhPrice', 'baseRentRange', 'pricetrend']
rentDf = immo_data[rentList]

In [32]:
rentDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   totalRent             228333 non-null  float64
 1   baseRent              268850 non-null  float64
 2   serviceCharge         261941 non-null  float64
 3   heatingCosts          85518 non-null   float64
 4   electricityBasePrice  46846 non-null   float64
 5   electricityKwhPrice   46846 non-null   float64
 6   baseRentRange         268850 non-null  int64  
 7   pricetrend            267018 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 16.4 MB


We notice that only baseRent and baseRentRange have values for all 268,850 records. We will determine the proportion of missing values for each feature.

In [61]:
# To create a dataframe with  %age of values that are missing
missing_rent = pd.concat([rentDf.isnull().sum(), 100 * rentDf.isnull().mean()], axis = 1)
missing_rent.columns = ['count', '%']
missing_rent.sort_values(by = 'count')

,count,%
baseRent,0,0.000000
baseRentRange,0,0.000000
pricetrend,1832,0.681421
serviceCharge,6909,2.569834
totalRent,40517,15.070485
heatingCosts,183332,68.191185
electricityBasePrice,222004,82.575414
electricityKwhPrice,222004,82.575414


We notice that electricityBasePrice and electrictyKwhPrice are missing over 82% of their values, limiting their use.

Let's see the number of unique values associated with these features.

In [33]:
rentDf.nunique()

totalRent               28486
baseRent                26659
serviceCharge           12266
heatingCosts             5669
electricityBasePrice        2
electricityKwhPrice        15
baseRentRange               9
pricetrend               1234
dtype: int64

In [41]:
rentDf.electricityBasePrice.unique() # Distinct values associated with the electricityBasePrice feature

array([  nan, 90.76, 71.43])

In [42]:
rentDf.electricityKwhPrice.unique() # Distinct values associated with the electricityKwhPrice feature

array([   nan, 0.1915, 0.2055, 0.1985, 0.1845, 0.2265, 0.2137, 0.2074,
       0.1775, 0.1705, 0.2125, 0.2276, 0.2144, 0.2205, 0.2132, 0.2195])

The dataset owner describes electricityBasePrice as the "monthly base price for electricity in € (deprecated since Feb 2020)" and electricityKwhPrice as the "electricity price per kwh (deprecated since Feb 2020)". 

In [48]:
rentDf[~ rentDf['electricityBasePrice'].isnull()]

,totalRent,baseRent,serviceCharge,heatingCosts,electricityBasePrice,electricityKwhPrice,baseRentRange,pricetrend
15,300.00,220.00,80.00,NaN,90.76,0.1915,1,1.67
17,325.00,200.00,50.00,75.00,90.76,0.2055,1,1.96
30,970.00,800.00,170.00,NaN,90.76,0.1985,5,6.49
31,445.18,315.18,130.00,NaN,90.76,0.1845,2,3.90
41,NaN,333.20,196.80,NaN,90.76,0.1985,2,1.82
...,...,...,...,...,...,...,...,...
268834,610.00,440.00,170.00,NaN,90.76,0.2055,3,1.92
268835,764.82,509.88,NaN,NaN,90.76,0.1985,4,3.57
268839,1980.00,1780.00,100.00,100.00,90.76,0.1985,8,7.43
268840,1479.64,1255.38,112.13,112.13,90.76,0.1985,7,6.90
